In [38]:
import os
import sys

sys.path.append(os.path.join(os.path.dirname(sys.path[0])))

from environs import Env
import pymongo
import requests
from tqdm import tqdm
import time

In [39]:
SEASON = 'f12' # 1-14 old, f1 - f13
PLAYLIST = 'ranked-doubles' # ranked-doubles, ranked-duels, ranked-standard
COUNT = 200
TIERS = range(21, 22) #(highest tier = 21)
PRO = 'true'

RATE_LIMIT_PER_HOUR = 490

env = Env()
env.read_env('.env')
TOKEN = os.environ["TEST_TOKEN"] # Ballchasing token
HEADERS = {"Authorization": TOKEN}
URL = "https://ballchasing.com/api/replays"

In [40]:
cluster = pymongo.MongoClient("mongodb://localhost:27017/")
database = cluster.get_database("rl_replays")

In [41]:
ranks = [
    "bronze-1",
    "bronze-2",
    "bronze-3",
    "silver-1",
    "silver-2",
    "silver-3",
    "gold-1",
    "gold-2",
    "gold-3",
    "platinum-1",
    "platinum-2",
    "platinum-3",
    "diamond-1",
    "diamond-2",
    "diamond-3",
    "champion-1",
    "champion-2",
    "champion-3",
    "grand-champion-1",
    "grand-champion-2",
    "grand-champion-3",
    "supersonic-legend"
]

In [42]:
delay = 3600 / RATE_LIMIT_PER_HOUR
time_exp = delay * COUNT * len(TIERS)

print(
    f"Time expected: {int(time_exp/3600)}hrs {int((time_exp%3600)/60)}min {int((time_exp%3600)%60)}sec"
)

for tier in TIERS:
    rank = ranks[tier]
    print(f"Starting {rank}[{tier}] rank")

    filters = {"count": COUNT, "playlist": PLAYLIST, "min-rank": rank, "max-rank": rank, 'season': SEASON, 'pro': PRO}

    response = requests.get(URL, headers=HEADERS, params=filters)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        break

    replays = response.json()["list"]
    data = tqdm(replays, desc=f"{rank}[{tier}]")
    for replay in data:
        replay_url = replay["link"]

        response = requests.get(replay_url, headers=HEADERS)
        if response.status_code != 200: continue

        replay_data = response.json()
        replay_id = replay_data["id"]

        if database.replays.find_one({"id": replay_id}): continue

        database.replays.insert_one(replay_data)
        time.sleep(delay)

    print(f"Finished {rank}[{tier}] rank")
    time.sleep(delay)

print("Finished")

Time expected: 0hrs 24min 29sec
Starting supersonic-legend[21] rank


supersonic-legend[21]: 100%|██████████| 200/200 [07:58<00:00,  2.39s/it]


Finished supersonic-legend[21] rank
Finished


In [43]:
# rank = ranks[0]
# filters = {"count": COUNT, "playlist": PLAYLIST, "min-rank": rank, "max-rank": rank, 'season': SEASON, 'pro': 'true'}
# response = requests.get(URL, headers=HEADERS, params=filters)
# replays = response.json()["list"]
# print(f"Total replays: {len(replays)}")
# replay = replays[0]
# replay